# Data Analysis

In [90]:
import os
import glob
import shutil
import numpy as np
from edd.data import IBMQData
import edd.workflow as wf

cwd = os.getcwd()
device = "Armonk"

base_dir = f"{cwd}/{device}/DelayData"

## Step 1: Load in Raw Data and Parse It

In [91]:
result_dir = f"{base_dir}/rawDelayData"
files = glob.glob(f"{result_dir}/*.yml")

In [92]:
len(files) / (6 * 10)

3.8666666666666667

In [93]:
sym_files = []
asym_files = []
for f in files:
    if "asym" in f:
        asym_files.append(f)
    else:
        sym_files.append(f)

In [94]:
def compute_fid(counts):
    """
    Computes fidelity as num0 / (num0 + num1).
    """
    num0 = counts.get('0x0', 0)
    num1 = counts.get('0x1', 0)
    
    return num0 / (num0 + num1)


#def compute_error(counts, resamples=1000):
    """
    Computes 2 \sigma error on fidelity.
    """
#    num0 = counts.get('0x0', 0)
#    num1 = counts.get('0x1', 0)
#    tot_counts = num0 + num1
#    prob0 = num0 / tot_counts
#    
#    fids = []
#    for j in range(resamples):
#        re_counts = np.random.choice([0, 1], size=tot_counts, p=[prob0, 1 - prob0]) 
#        fid = 1 - sum(re_counts) / tot_counts
#        fids.append(fid)
#        
#    return 2 * np.std(fids)

def compute_error(counts, resamples=1000):
    """
    Faster way compute errors.
    """
    return 0.0033

## Step 1: Summarize symmetric sequence data

In [95]:
# armonk sequences
list_of_seq = ["qdd_1_5", "ur20", "xy4", "qdd_1_3", "uddx10", "super_cpmg", "ur10", "super_hahn", "purex", "free"]

In [71]:
# bogota sequences
list_of_seq = ["qdd_1_4", "qdd_1_2", "ur20", "cdd3", "rga64c", "rga32c", "xy4", "super_hahn", "purex", "free"]

In [49]:
# jakarta sequences
list_of_seq = ["qdd_1_4", "ur10", "rga64c", "ur6", "ur50", "qdd_1_6", "xy4", "super_hahn", "purex", "free"]

In [96]:
data_dict = {}
for seq in list_of_seq:
    data = IBMQData(None, seq)
    # find corresponding file and load it
    file_idxs = np.nonzero([seq in file for file in sym_files])[0]
    count = 0
    for fidx in file_idxs:
        data.load_raw_data(sym_files[fidx])
        # format the data in way easier to parse
        data.format_data()
        for tup in data.tuple_data:
            name, _, counts = tup
            # extract info from name
            split = name.split("_")
            delay_idx = split.index("delay") + 1
            delay = float(split[delay_idx])
            state_idx = split.index("state") + 1
            state = int(split[state_idx])
            fid = compute_fid(counts)
            error = compute_error(counts, 250)
            data_dict[(seq, state, delay, count)] = (fid, error)
        count += 1

## Step 2: Make the CSV Files for symmetric sequence data

In [97]:
for seq in list_of_seq:
    for state in range(6):
        file_prefix = f"{base_dir}/csvDelayData/delayData_{device}_seq_{seq}_state_{state}_sym_True"
        desired_key = (seq, state)
        delays = {}
        fids = {}
        errs = {}
        counts = []
        for key, fid_tup in data_dict.items():
            if key[0] == desired_key[0] and key[1] == desired_key[1]:
                count = key[-1]
                if count not in counts:
                    counts.append(count)
                    delays[count] = [key[2]]
                    fids[count] = [fid_tup[0]]
                    errs[count] = [fid_tup[1]]
                else:
                    delays[count].append(key[2])
                    fids[count].append(fid_tup[0])
                    errs[count].append(fid_tup[1])

        for c in counts:
            fname = f"{file_prefix}_count_{c}.csv"
            with open(fname, 'w') as f:
                for j in range(len(delays[c])):
                    f.write(f"{delays[c][j]},{fids[c][j]},{errs[c][j]}\n")

## Step 3: Summarize asymmereic sequence data

In [98]:
data_dict = {}
for seq in list_of_seq:
    data = IBMQData(None, seq)
    # find corresponding file and load it
    file_idxs = np.nonzero([seq in file for file in asym_files])[0]
    count = 0
    for fidx in file_idxs:
        data.load_raw_data(asym_files[fidx])
        # format the data in way easier to parse
        data.format_data()
        for tup in data.tuple_data:
            name, _, counts = tup
            # extract info from name
            split = name.split("_")
            delay_idx = split.index("delay") + 1
            delay = float(split[delay_idx])
            state_idx = split.index("state") + 1
            state = int(split[state_idx])
            fid = compute_fid(counts)
            error = compute_error(counts, 250)
            data_dict[(seq, state, delay, count)] = (fid, error)
        count += 1

## Step 4: Make the CSV Files for asymmetric sequence data

In [99]:
for seq in list_of_seq:
    for state in range(6):
        file_prefix = f"{base_dir}/csvDelayData/delayData_{device}_seq_{seq}_state_{state}_sym_False"
        desired_key = (seq, state)
        delays = {}
        fids = {}
        errs = {}
        counts = []
        for key, fid_tup in data_dict.items():
            if key[0] == desired_key[0] and key[1] == desired_key[1]:
                count = key[-1]
                if count not in counts:
                    counts.append(count)
                    delays[count] = [key[2]]
                    fids[count] = [fid_tup[0]]
                    errs[count] = [fid_tup[1]]
                else:
                    delays[count].append(key[2])
                    fids[count].append(fid_tup[0])
                    errs[count].append(fid_tup[1])

        for c in counts:
            fname = f"{file_prefix}_count_{c}.csv"
            with open(fname, 'w') as f:
                for j in range(len(delays[c])):
                    f.write(f"{delays[c][j]},{fids[c][j]},{errs[c][j]}\n")

## Step 5: Turn _ into - for sequence names in csv handles

In [100]:
files = glob.glob(f"{base_dir}/csvDelayData/*.csv")
import os

In [101]:
for f in files:
    split_f = f.split("_")
    seq_idx = split_f.index("seq") + 1
    seq = split_f[seq_idx]
    if seq == "qdd":
        full_seq = f"{seq}-{split_f[seq_idx + 1]}-{split_f[seq_idx + 2]}"
        split_f[seq_idx] = full_seq
        split_f[seq_idx + 1] = None
        split_f[seq_idx + 2] = None
        split_f.remove(None)
        split_f.remove(None)
        
    if seq == "super":
        full_seq = f"{seq}-{split_f[seq_idx + 1]}"
        split_f[seq_idx] = full_seq
        split_f[seq_idx + 1] = None
        split_f.remove(None)
    new_f = "_".join(split_f)
    
    os.rename(f, new_f)

In [69]:
data_dir = "/Users/worknic/Documents/Armonk_Bogota_24AugData/Bogota/q3/min_delay/"

for file in data_files:
    file = file + ".csv"
    shutil.copy(file, data_dir)

NameError: name 'data_files' is not defined

In [103]:
[x for x in range(10)][0:4]

[0, 1, 2, 3]

In [104]:
[x for x in range(10)][4:8]

[4, 5, 6, 7]